In [1]:
# 抓取資料
!wget -c https://ai100-fileentity.cupoy.com/ml100/homework/data/1582795766653/titanic_test.csv?t=1582795815962 -O titanic_test.csv
!wget -c https://ai100-fileentity.cupoy.com/ml100/homework/data/1582795766653/titanic_train.csv?t=1582795819472 -O titanic_train.csv

--2020-07-20 11:37:02--  https://ai100-fileentity.cupoy.com/ml100/homework/data/1582795766653/titanic_test.csv?t=1582795815962
Resolving ai100-fileentity.cupoy.com (ai100-fileentity.cupoy.com)... 52.84.16.69, 52.84.16.99, 52.84.16.48, ...
Connecting to ai100-fileentity.cupoy.com (ai100-fileentity.cupoy.com)|52.84.16.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28210 (28K) [application/octet-stream]
Saving to: ‘titanic_test.csv’

titanic_test.csv    100%[===================>]  27.55K  --.-KB/s    in 0.05s   

2020-07-20 11:37:02 (591 KB/s) - ‘titanic_test.csv’ saved [28210/28210]

--2020-07-20 11:37:05--  https://ai100-fileentity.cupoy.com/ml100/homework/data/1582795766653/titanic_train.csv?t=1582795819472
Resolving ai100-fileentity.cupoy.com (ai100-fileentity.cupoy.com)... 52.84.16.48, 52.84.16.69, 52.84.16.11, ...
Connecting to ai100-fileentity.cupoy.com (ai100-fileentity.cupoy.com)|52.84.16.48|:443... connected.
HTTP request sent, awaiting response... 

In [5]:
import pandas
import numpy as np

df_titanic_train=pandas.read_csv('titanic_train.csv')
df_titanic_test=pandas.read_csv('titanic_test.csv')

df_titanic_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df_titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [37]:
# 重組資料成為訓練 / 預測用格式
# Y of training set
train_Y=df_titanic_train['Survived']

idlist=df_titanic_test['PassengerId']
df_train=df_titanic_train.drop(['PassengerId','Survived'],axis=1)
df_test=df_titanic_test.drop(['PassengerId'],axis=1)

# 合併
df=pandas.concat([df_train,df_test])
df.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# 秀出資料欄位的類型與數量 - 使用groupby
dftypes=df.dtypes.reset_index()
dftypes.columns=['count','type']
dftypes.groupby('type').aggregate('count').reset_index()


,type,count
0,int64,3
1,float64,2
2,object,5


In [22]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中

int_features = []
float_features = []
object_features = []

for type,feature in zip(df.dtypes,df.columns):
  if type=='int64':
    int_features.append(feature)
  elif type=='float64':
    float_features.append(feature)
  elif type=='object':
    object_features.append(feature)

print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')


3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


#作業1
試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?

object type是無法產生mean之類計算結果為實數的操作

#作業2
思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [23]:
df[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

# 依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作

In [26]:
df[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [27]:
df[float_features].mean()

Age     29.881138
Fare    33.295479
dtype: float64

In [28]:
df[object_features].mean()

Series([], dtype: float64)

In [29]:
df[int_features].max()

Pclass    3
SibSp     8
Parch     9
dtype: int64

In [30]:
df[float_features].max()

Age      80.0000
Fare    512.3292
dtype: float64

In [31]:
df[object_features].max()

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

In [33]:
df[int_features].nunique()

Pclass    3
SibSp     7
Parch     8
dtype: int64

In [34]:
df[float_features].nunique()

Age      98
Fare    281
dtype: int64

In [35]:
df[object_features].nunique()

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64